<H1>AML Project 2: Decision Trees, SVMs and Ensemble Learning</H1>
<p1>Andrew J Markland, ajm259@uakron.edu</p1> <br>
<p1>The University of Akron: CEPS : School of Computer Science</p1><br>
<p1>Applied Machine Learning: CPSC:436:010</p1><br>
<p1>Dr. Zhong-Hui Duan </p1><br><br>
<p1>Abstract: The purpose of this assingment is to display understanding and untilization of Decesion tree, ensemble learning</p1><br>
<p1>and SVM's through predicting the likelyhood of strokes through models that we will train with data retrived from NHANES</p1><br><br>
<p1>Disclaimer** While this project is public in my public repository on github, I do not discourage others from using it as a </p1><br>
<p1>reference for their own implementations, but the work written here is expressly written and owned by Andrew J Markland.</p1><br>
<p1>I do not condone nor tolerate academic dishonesty, and should the question of plagerism arise derived from my works</p1><br>
<p1>copies of this repository shall be promptly submitted to the assigning professor and the department of student affairs.</p1>